In [2]:
import os, os.path
import numpy as np
from time import time
import pandas as pd
import math as m
import setup_runs as sr
import importlib
import matplotlib.pyplot as plt
import itertools

In [14]:

dict_2.keys()

dict_keys(['Agriculture', 'Forestry', 'Ippu', 'Waste', 'commercial', 'electricity_generation', 'industry_and_mining', 'public', 'residential', 'transport'])

In [20]:
disp = list(dict_2["electricity_generation"][dict_2["electricity_generation"]["x_or_xl"].isin(["x", "xl"])]["parameter"])
disp.sort()
disp

['fuel_price_coal',
 'fuel_price_coal',
 'fuel_price_diesel',
 'fuel_price_diesel',
 'fuel_price_fuel_oil',
 'fuel_price_fuel_oil',
 'fuel_price_natural_gas',
 'fuel_price_natural_gas',
 'hydrology_exceedance_probability',
 'investment_cost_csp_solar',
 'investment_cost_csp_solar',
 'investment_cost_geothermal',
 'investment_cost_geothermal',
 'investment_cost_natural_gas_cc',
 'investment_cost_natural_gas_cc',
 'investment_cost_pv_solar',
 'investment_cost_pv_solar',
 'investment_cost_wind',
 'investment_cost_wind']

In [48]:

#df_attribute_master_id = pd.read_csv(sr.fp_csv_attribute_master)
#pull in parameters
df_params = pd.read_csv(sr.fp_csv_parameter_ranges)
#

dict_1 = pd.read_excel(os.path.join(sr.dir_ref, "parameter_x_xl_designation", "parameter_x-xl_designation.xlsx"), sheet_name = None)
dict_2 = pd.read_excel(os.path.join(sr.dir_ref, "parameter_x_xl_designation", "parameter_x-xl_designation_carlos.xlsx"), sheet_name = None)

keys_1 = ["Agriculture", "Forestry", "Ippu", "Waste"]
keys_2 = [x for x in dict_2.keys() if x not in keys_1]

params_xl = set({})
for k in dict_1.keys():
    if k in keys_1:
        params_xl = params_xl | set(dict_1[k][dict_1[k]["x_or_xl"] != "x"]["parameter"])
    else:
        params_xl = params_xl | set(dict_2[k][dict_2[k]["x_or_xl"] != "x"]["parameter"])

        

##  SOME FUNCTIONS

def check_param(param):
    if param in params_xl:
        out = "xl"
    else:
        out = "x"
        
    #check trajgroup
    if ("trajgroup_" in param) & ("-lhs" in param):
        out = "xl"
        
    return out

def disp_str(str_in):
    str_out = str_in.replace("_", " ").replace("-", " ")
    str_out = str_out.split(" ")
    
    s_new = ""
    for s in str_out:
        if len(s) > 1:
            s = s[0].upper() + s[1:len(s)].lower()
        else:
            s = s.upper()
        s_new = s_new + " " + s
    s_new = s_new[1:len(s_new)]
    
    return s_new



##  INITIALIZE ATTRIBUTE

fields_merge = ["parameter", "sector"]
st_tuples = [tuple(x) for x in np.array(df_params[["strategy_id", "time_series_id"]].drop_duplicates())]

#initialize the parameter attribute
df_attr_pr = df_params[fields_merge].drop_duplicates()
#grab fields with "trajmax/trajmin/trajmix"

#get string identifier
substr_max = "trajmax"
substr_min = "trajmin"
substr_mix = "trajmix"
substr_group = "trajgroup"
#get parameters that should be mixed
params_mix_loop_0 = [x for x in list(df_attr_pr["parameter"]) if ((substr_mix + "_") in x)]
#get dictionary to map to sector
dict_pml_sector = {}

df_pml_append = []

for pml in params_mix_loop_0:
    #get the sector
    sec = str(df_attr_pr[df_attr_pr["parameter"] == pml]["sector"].iloc[0])
    
    #get all parameters to drop
    params_pml_drop = [pml, pml.replace(substr_mix, substr_max), pml.replace(substr_mix, substr_min)]
    df_attr_pr = df_attr_pr[~df_attr_pr["parameter"].isin(params_pml_drop)]
     
    #check if this parameter needs to be added to xl list
    pml_add_pxl = pml.replace(substr_mix + "_", "")
    if pml_add_pxl in params_xl:
        params_xl = params_xl | set({pml_add_pxl})
    #check if group is specified
    if not substr_group in pml:
        #append to the new dataframe
        df_pml_append.append([pml, sec])

df_pml_append = pd.DataFrame(df_pml_append, columns = fields_merge)



#build new attribute table
df_attr_pr = pd.concat([df_attr_pr[~df_attr_pr["parameter"].isin(df_pml_append["parameter"])], df_pml_append[list(df_attr_pr.columns)]], axis = 0)
#add in "x/xl" designation
df_attr_pr["x_xl_designation"] = [check_param(x) for x in df_attr_pr["parameter"]]
#reorder
df_attr_pr = df_attr_pr.sort_values(by = ["sector", "parameter"]).reset_index(drop = True)
#add display string
df_attr_pr["display"] = [disp_str(x) for x in list(df_attr_pr["parameter"])]

#map of replacements for selected parameters:
dict_repl_disp = {
    "Trajgroup 1 Lhs": "Copper Heat Energy - % Towards Optimistic",
    "Trajgroup 2 Lhs": "Open Pit Mine Energy - % Towards Optimistic",
    "Trajgroup 3 Lhs": "Other Industries Heat Energy - % Towards Optimistic",
    "Trajgroup 4 Lhs": "Residential Apartment ACS Energy - % Towards Optimistic",
    "Trajgroup 5 Lhs": "Residential House ACS Energy - % Towards Optimistic",
    "Trajgroup 6 Lhs": "Private Transportation Electrification - % Towards Optimistic",
    "Trajgroup 7 Lhs": "Hydrogen Adoption in Trucks - % Towards Optimistic"
}

#add in trajmix displays
dict_add = dict([[x, x.replace("Trajmix ", "").replace("Pib", "PIB")] for x in df_attr_pr["display"] if "Trajmix" in x])

for k in dict_add.keys():
    dict_repl_disp.update({k: dict_add[k]})
    
df_attr_pr["display"] = df_attr_pr["display"].replace(dict_repl_disp)

df_attr_pr.to_csv(sr.fp_csv_attribute_parameters, index = None, encoding = "UTF-8")





In [44]:
k = "industry_and_mining"
if k in keys_1:
    px = set(dict_1[k][dict_1[k]["x_or_xl"] != "x"]["parameter"])
else:
    px = set(dict_2[k][dict_2[k]["x_or_xl"] != "x"]["parameter"])

In [49]:
params_mix_loop_0

['trajmix_pib',
 'trajmix_poblacion',
 'trajmix_incertidumbre_megaincendio',
 'trajmix_fuel_price_coal',
 'trajmix_fuel_price_diesel',
 'trajmix_fuel_price_fuel_oil',
 'trajmix_fuel_price_natural_gas',
 'trajmix_investment_cost_csp_solar',
 'trajmix_investment_cost_geothermal',
 'trajmix_investment_cost_natural_gas_cc',
 'trajmix_investment_cost_pv_solar',
 'trajmix_investment_cost_wind',
 'trajgroup_1-trajmix_copper_heat_diesel',
 'trajgroup_1-trajmix_copper_heat_solar',
 'trajgroup_2-trajmix_copper_open_pit_mine_diesel',
 'trajgroup_2-trajmix_copper_open_pit_mine_electricitiy',
 'trajgroup_3-trajmix_other_industries_heat_coal',
 'trajgroup_3-trajmix_other_industries_heat_diesel',
 'trajgroup_3-trajmix_other_industries_heat_fuel_oil',
 'trajgroup_3-trajmix_other_industries_heat_natural_gas',
 'trajgroup_3-trajmix_other_industries_heat_pliqgas',
 'trajgroup_3-trajmix_other_industries_heat_solar',
 'trajmix_copper_production',
 'trajmix_iron_production',
 'trajmix_pulp_production',
 'tr

In [50]:
df_attr_pr[["trajmix" in x for x in df_attr_pr["parameter"]]]

,parameter,sector,x_xl_designation,display
12,trajmix_pib,Analytica_all,x,PIB
13,trajmix_poblacion,Analytica_all,x,Poblacion
27,trajmix_incertidumbre_megaincendio,Analytica_forestry,x,Incertidumbre Megaincendio
131,trajmix_fuel_price_coal,electricity_generation,x,Fuel Price Coal
132,trajmix_fuel_price_diesel,electricity_generation,x,Fuel Price Diesel
133,trajmix_fuel_price_fuel_oil,electricity_generation,x,Fuel Price Fuel Oil
134,trajmix_fuel_price_natural_gas,electricity_generation,x,Fuel Price Natural Gas
135,trajmix_investment_cost_csp_solar,electricity_generation,x,Investment Cost Csp Solar
136,trajmix_investment_cost_geothermal,electricity_generation,x,Investment Cost Geothermal
137,trajmix_investment_cost_natural_gas_cc,electricity_generation,x,Investment Cost Natural Gas Cc


In [51]:
 params_xl

{'cement_frac_biomass',
 'cement_frac_coal',
 'cement_frac_diesel',
 'cement_frac_electric',
 'cement_frac_hydrogen',
 'cement_frac_kerosene',
 'cement_frac_natural_gas',
 'cement_frac_solar',
 'cement_frac_thermal_solar',
 'commercial_frac_biomass',
 'commercial_frac_diesel',
 'commercial_frac_electric',
 'commercial_frac_hydrogen',
 'commercial_frac_kerosene',
 'commercial_frac_natural_gas',
 'commercial_frac_pliqgas',
 'commercial_frac_solar',
 'commercial_heating_diesel',
 'commercial_heating_electric',
 'commercial_heating_natural_gas',
 'commercial_heating_pliqgas',
 'commercial_other_biomass',
 'commercial_other_diesel',
 'commercial_other_electric',
 'commercial_other_gas',
 'commercial_other_hydrogen',
 'commercial_other_kerosene',
 'commercial_other_natural_gas',
 'commercial_other_pliqgas',
 'copper_heat_electricitiy',
 'copper_heat_hydrogen',
 'copper_heat_natural_gas',
 'copper_heat_plqgas',
 'copper_motor_diesel',
 'copper_motor_electricitiy',
 'copper_motor_hydrogen',
 '